In [1]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)


In [7]:
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

#Define a new graph
workflow = StateGraph(state_schema=MessagesState)


#Define the function that calls the model
def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    # Update message history with response:
    return {"messages": response}


#Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

#Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [8]:
config = {"configurable": {"thread_id": "test_22"}}

In [9]:
class HumanAttributes:
    def __init__(self):
        # Initialize the 'map' attribute as a dictionary where all attributes are keys with empty strings as default values
        self.map = {
            "Name": "",
            "Age": "",
            "Gender": "",
            "Nationality": "",
            "Profession": "",
            "Education": "",
            "Languages Spoken": "",
            "Hobbies": "",
            "Personality Traits": "",
            "Values": "",
            "Beliefs": "",
            "Favorite Books": "",
            "Favorite Movies": "",
            "Life Goals": "",
            "Dreams": "",
            "Bucket List": "",
            "Relationships": "",
            "Current Emotional State": "",
        }

    def set_attribute(self, key, value):
        """Set an attribute dynamically in the map."""
        if key in self.map:
            self.map[key] = str(value)  # Convert value to string before storing
        else:
            print(f"❗ Attribute '{key}' not found.")

    def get_attribute(self, key):
        """Get the value of a specific attribute from the map."""
        return self.map.get(key, "Unknown")

    def summary(self):
        """Return a full summary of the attributes from the map."""
        for key, value in self.map.items():
            if value:  # If the value is an empty string
                print("the attribute is: ", key, "and the value is: ", value)

    def __str__(self):
        """Return a pretty-printed string summary of the attributes in the map."""
        return "\n".join([f"{key}: {value or 'Unknown'}" for key, value in self.map.items()])

    def missing_attributes(self):
        missing = []
        for key, value in self.map.items():
            if not value:  # If the value is an empty string
                missing.append(key)

        if not missing:
            return "All attributes have been filled out."
        else:
            return f"❗ The following attributes are still missing: {', '.join(missing)}"



In [7]:
from langchain.prompts import ChatPromptTemplate

system_prompt = """ 

    My HumanAttributes class has the following attributes "Name", "Age", Gender", "Nationality", "Profession", "Education", "Languages Spoken", "Hobbies"
            ,"Personality Traits", "Values", "Beliefs", "Favorite Books", "Favorite Movies", "Life Goals", "Dreams", "Bucket List", "Relationships", "Current Emotional State"

   You are a friendly and inquisitive AI chat bot. I want you to remember all the attributes for this class and fill it up as you find it. Ask questions to get answers about the human and
   when you do, find out other attributes in the next question. Remember to keep your reply to a maximum of six lines whilst having only one question at a time. Also, don't be robotic.

   **Rules**:
    1. Be conversational and friendly.
    2. Ask only one question at a time if asking a question.
    3. Only ask for missing information.
    4. Never repeat questions.
    5. Adapt based on their responses.
    6. Limit questions and conversation to a maximum of three lines.
    7. Don't deviate to much from your question and answers.

    Make sure to address the human properly.
"""

app.invoke({"messages": [HumanMessage(content=system_prompt)]}, config=config)

{'messages': [HumanMessage(content=' \n\n    My HumanAttributes class has the following attributes "Name", "Age", Gender", "Nationality", "Profession", "Education", "Languages Spoken", "Hobbies"\n            ,"Personality Traits", "Values", "Beliefs", "Favorite Books", "Favorite Movies", "Life Goals", "Dreams", "Bucket List", "Relationships", "Current Emotional State"\n\n   You are a friendly and inquisitive AI chat bot. I want you to remember all the attributes for this class and fill it up as you find it. Ask questions to get answers about the human and\n   when you do, find out other attributes in the next question. Remember to keep your reply to a maximum of six lines whilst having only one question at a time. Also, don\'t be robotic.\n\n   **Rules**:\n    1. Be conversational and friendly.\n    2. Ask only one question at a time if asking a question.\n    3. Only ask for missing information.\n    4. Never repeat questions.\n    5. Adapt based on their responses.\n    6. Limit ques

In [ ]:
import requests
from flask import Flask, request, jsonify
from langchain.schema import HumanMessage

jupyter_app = Flask(__name__)

human = HumanAttributes()

@jupyter_app.route('/trigger-feedback', methods=['POST'])
def trigger_feedback():
    try:
        data = request.get_json()
        if not data or 'answer' not in data:
            return jsonify({'error': 'Missing answer in request'}), 400
        
        user_message = data['answer']

        missing_attributes = "The user is about to send a message. Converse  and be friendly with them, keeping in mind the missing attributes. Find out only one attribute from a statement, without making it so blatant. Be conversational and do not sound robotic." + human.missing_attributes()
        
        # Let the LLM know there is an incoming user message.
        
        final_message = missing_attributes + "The sentence after this is the user message enter an appropriate message to the user based on my prompting." + user_message
        print(final_message)
        # Process message through LangGraph
        response_to_user = app.invoke(
            {"messages": [HumanMessage(content=final_message)]},
            config=config
        )

        attribute = app.invoke(
            {"messages": [HumanMessage(content="Based on the user message, and the HumanAttributes class write a single string of the attribute. For eg like Name and Age. Do not include quotations or punctuations.")]},
            config=config
        )

        value_message = "So what is the user's " + attribute["messages"][-1].content

        value = app.invoke(
            {"messages": [HumanMessage(content=value_message)]},
            config=config
        )
        
        human.set_attribute(attribute["messages"][-1].content, value["messages"][-1].content)

        print(attribute["messages"][-1].content, " ", value["messages"][-1].content)

        # Extract the last message content
        feedback = response_to_user["messages"][-1].content

        filled = False

        if human.missing_attributes == "All attributes have been filled out.":
            filled = True
        
        return jsonify({
            'feedback': feedback,
            'status': 'success',
            'filled': filled
        })
        
    except Exception as e:
        print(f"Error generating feedback: {str(e)}", flush=True)
        return jsonify({'error': str(e)}), 500
        
# Run the server in a separate thread to avoid blocking the notebook
from threading import Thread
def run_jupyter_server():
    jupyter_app.run(host='127.0.0.1', port=5001, debug=False)

if __name__ == '__main__':
    # Start the server in a thread
    server_thread = Thread(target=run_jupyter_server)
    server_thread.daemon = True  # Stops when notebook stops
    server_thread.start()
    print("Jupyter question generator server running on port 5001...")

Jupyter question generator server running on port 5001...
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit


The user is about to send a message. Converse  and be friendly with them, keeping in mind the missing attributes. Find out only one attribute from a statement, without making it so blatant. Be conversational and do not sound robotic.❗ The following attributes are still missing: Name, Age, Gender, Nationality, Profession, Education, Languages Spoken, Hobbies, Personality Traits, Values, Beliefs, Favorite Books, Favorite Movies, Life Goals, Dreams, Bucket List, Relationships, Current Emotional StateThe sentence after this is the user message enter an appropriate message to the user based on my prompting.hello


127.0.0.1 - - [06/Apr/2025 01:53:32] "POST /trigger-feedback HTTP/1.1" 200 -


Profession   I don't know the user's profession. I'm waiting for them to tell me! I'm just trying to get to know them better through conversation.
The user is about to send a message. Converse  and be friendly with them, keeping in mind the missing attributes. Find out only one attribute from a statement, without making it so blatant. Be conversational and do not sound robotic.❗ The following attributes are still missing: Name, Age, Gender, Nationality, Education, Languages Spoken, Hobbies, Personality Traits, Values, Beliefs, Favorite Books, Favorite Movies, Life Goals, Dreams, Bucket List, Relationships, Current Emotional StateThe sentence after this is the user message enter an appropriate message to the user based on my prompting.I'm currently a part of a hackathon


...

Favorite Books   I don't know the user's favorite books. I asked a question that *might* lead them to reveal something about their favorite books, but I'm still waiting for their response.
The user is about to send a message. Converse  and be friendly with them, keeping in mind the missing attributes. Find out only one attribute from a statement, without making it so blatant. Be conversational and do not sound robotic.❗ The following attributes are still missing: Name, Age, Gender, Nationality, Education, Languages Spoken, Beliefs, Favorite Movies, Dreams, Bucket List, Relationships, Current Emotional StateThe sentence after this is the user message enter an appropriate message to the user based on my prompting.I don't have any favorite books, I would love to learn things for fun if I had time


127.0.0.1 - - [06/Apr/2025 01:56:56] "POST /trigger-feedback HTTP/1.1" 200 -


Education   I don't know the user's specific educational background, but based on their interest in learning things for fun and their role as a front-end developer in a hackathon, I can infer they likely have some level of formal or informal education in computer science or a related field. However, I don't know the specifics of their education.
The user is about to send a message. Converse  and be friendly with them, keeping in mind the missing attributes. Find out only one attribute from a statement, without making it so blatant. Be conversational and do not sound robotic.❗ The following attributes are still missing: Name, Age, Gender, Nationality, Languages Spoken, Beliefs, Favorite Movies, Dreams, Bucket List, Relationships, Current Emotional StateThe sentence after this is the user message enter an appropriate message to the user based on my prompting.Philosophy, Computer Science


127.0.0.1 - - [06/Apr/2025 01:57:11] "POST /trigger-feedback HTTP/1.1" 200 -


Beliefs   I don't know the user's beliefs. I asked a question that *might* lead them to reveal something about their beliefs, but I'm still waiting for their response.


In [16]:
print(human.summary())
print(human.missing_attributes())

the attribute is:  Name and the value is:  Ramana
the attribute is:  Age and the value is:  18
the attribute is:  Gender and the value is:  Male
the attribute is:  Nationality and the value is:  India
the attribute is:  Profession and the value is:  Student
the attribute is:  Education and the value is:  Studying in University of Illinois Urbana Champaign
the attribute is:  Languages Spoken and the value is:  Hindi and Tamil
the attribute is:  Hobbies and the value is:  Playing Tennis and coding
the attribute is:  Personality Traits and the value is:  Introversion
the attribute is:  Values and the value is:  Freedom and passion
the attribute is:  Beliefs and the value is:  Agnostic
the attribute is:  Favorite Books and the value is:  Organic chemistry from highschool
the attribute is:  Favorite Movies and the value is:  Action movies
the attribute is:  Life Goals and the value is:  Stretching
the attribute is:  Dreams and the value is:  Own a fast fast car
the attribute is:  Bucket Lis